# Ceneo Scraper

# Biblioteki

In [1]:
import requests

## Wysłanie żądania dostępu do zasobu do serwera

In [3]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-2"
response = requests.get(url)


200

In [4]:
import requests
from bs4 import BeautifulSoup

In [ ]:
page_dom = BeautifulSoup(response.text, "html.parser")